In [18]:
from mpnn_train import network
import torch.optim as optim
from torch import nn
import torch

# hannas stark..spain germany...switerzland....dcn
# graph net network in action..........matrix in 3d........brzil


In [16]:
# network
optimizer = optim.Adam(network.parameters(), lr=0.001)
criterion = torch.nn.KLDivLoss()

In [25]:
from ashishcode import load_molecule,MolecularGraph,params
path = "data\pre-training\gdb13_1K\Train.smi"
molecule_set = load_molecule(path)
print(molecule_set)
for mol in molecule_set:
    molecule = MolecularGraph(mol)
    # print(molecule)

True


In [26]:
molecule.get_decoding_route_state(3)

degree is  1   [array([1, 3, 7], dtype=int64)]
here is the value  [array([1, 3, 7], dtype=int64)]
degree is  1   [array([ 8, 10], dtype=int64)]
here is the value  [array([ 8, 10], dtype=int64)]
[0 0 0 0 0 0 0 0 0]
[] node index is as   12


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:

for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):#this we have to find......

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [3]:
from graphinvent import gnn
import torch
from collections import namedtuple

In [4]:
hyperparameters = {
        "mlp1_depth"          : 4,
        "mlp1_dropout_p"      : 0.0,
        "mlp1_hidden_dim"     : 100,
        "mlp2_depth"          : 4,
        "mlp2_dropout_p"      : 0.0,
        "mlp2_hidden_dim"     : 100,
        "hidden_node_features": 100,
        "message_passes"      : 3,
        "message_size"        : 100,
        "n_edge_features"     :4
    }
import json
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
constants = dotdict(hyperparameters)


In [5]:
constants.n_edge_features

4

In [6]:
from ashishcode import load_molecule,MolecularGraph,params
path = "data\pre-training\gdb13_1K\Train.smi"
molecule_set = load_molecule(path)
print(molecule_set)
for mol in molecule_set:
    molecule = MolecularGraph(mol)
    print(molecule)
    break




True


In [7]:

class SummationMPNN(torch.nn.Module):
    """
    Abstract `SummationMPNN` class. Specific models using this class are
    defined in `mpnn.py`; these are MNN, S2V, and GGNN.
    """
    def __init__(self, constants : namedtuple):

        super().__init__()

        self.hidden_node_features = constants.hidden_node_features
        self.edge_features        = constants.n_edge_features
        self.message_size         = constants.message_size
        self.message_passes       = constants.message_passes
        self.constants            = constants

    def forward(self, nodes : torch.Tensor, edges : torch.Tensor) -> None:
        adjacency = torch.sum(edges, dim=3)

        # **note: "idc" == "indices", "nghb{s}" == "neighbour(s)"
        (edge_batch_batch_idc,
         edge_batch_node_idc,
         edge_batch_nghb_idc) = adjacency.nonzero(as_tuple=True)
        print("sizes are edge_batch_node_idc ",edge_batch_node_idc)

        (node_batch_batch_idc, node_batch_node_idc) = adjacency.sum(-1).nonzero(as_tuple=True)

        same_batch = node_batch_batch_idc.view(-1, 1) == edge_batch_batch_idc
        same_node  = node_batch_node_idc.view(-1, 1) == edge_batch_node_idc

        # element ij of `message_summation_matrix` is 1 if `edge_batch_edges[j]`
        # is connected with `node_batch_nodes[i]`, else 0
        message_summation_matrix = (same_batch * same_node).float()

        edge_batch_edges = edges[edge_batch_batch_idc, edge_batch_node_idc, edge_batch_nghb_idc, :]

        # pad up the hidden nodes
        hidden_nodes = torch.zeros(nodes.shape[0],
                                   nodes.shape[1],
                                   self.hidden_node_features,
                                   device=self.constants.device)
        hidden_nodes[:nodes.shape[0], :nodes.shape[1], :nodes.shape[2]] = nodes.clone()
        node_batch_nodes = hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :]
        

        for _ in range(self.message_passes):
            edge_batch_nodes = hidden_nodes[edge_batch_batch_idc, edge_batch_node_idc, :]

            edge_batch_nghbs = hidden_nodes[edge_batch_batch_idc, edge_batch_nghb_idc, :]

            print("hello ji ",edge_batch_nghbs.shape,edge_batch_nodes.shape,hidden_nodes.shape)

            message_terms = self.message_terms(edge_batch_nodes,
                                                  edge_batch_nghbs,
                                                  edge_batch_edges)

            if len(message_terms.size()) == 1:  # if a single graph in batch
                message_terms = message_terms.unsqueeze(0)

            # the summation in eq. 1 of the NMPQC paper happens here
            messages = torch.matmul(message_summation_matrix, message_terms)

            node_batch_nodes = self.update(node_batch_nodes, messages)
            hidden_nodes[node_batch_batch_idc, node_batch_node_idc, :] = node_batch_nodes.clone() #updated the hidden states

        node_mask = adjacency.sum(-1) != 0
        output    = self.readout(hidden_nodes, nodes, node_mask)

        return output


In [57]:
import numpy as np
def pad(node,edge)->np.array:
    n_size = node.shape
    nodes_pad = np.zeros((13,9))
    nodes_pad[:n_size[0],:n_size[1]] = node

    #import numpy as np
    e_size = edge.shape
    edge_pad = np.zeros((13,13,4))
    edge_pad[:e_size[0],:e_size[1],:] = edge

    return nodes_pad,edge_pad

In [69]:
network = MNN(constants)
node,edge = molecule.get_graph_state()
node,edge = pad(node,edge)#now we have padeed them according to max lenght
nodes,edges = torch.Tensor(node).view((1,13,9)),torch.Tensor(edge).view((1,13,13,4))
output = network(nodes,edges)
print(out)


100 100 4
torch.Size([100, 100, 4])
sizes are edge_batch_node_idc  tensor([0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9])
hello ji  torch.Size([24, 100]) torch.Size([24, 100]) torch.Size([1, 13, 100])
hello ji  torch.Size([24, 100]) torch.Size([24, 100]) torch.Size([1, 13, 100])
hello ji  torch.Size([24, 100]) torch.Size([24, 100]) torch.Size([1, 13, 100])
input to network  torch.Size([1, 13, 100])
api is  torch.Size([1, 1, 100])
dims fadd ip torch.Size([1, 13, 100]) torch.Size([1, 1, 100]) torch.Size([1, 14, 100])
global readout shapes  torch.Size([1, 1, 14, 100]) torch.Size([1, 1, 14, 100]) torch.Size([1, 1, 100])
final shape  torch.Size([1, 29, 100])


In [73]:
network = MNN(constants)
node,edge = molecule.get_graph_state()
# nodes,edges = torch.Tensor(node).view((1,13,9)),torch.Tensor(edge).view((1,13,13,4))
network(nodes,edges)

100 100 4
torch.Size([100, 100, 4])
sizes are edge_batch_node_idc  tensor([0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 9, 9])
hello ji  torch.Size([24, 100]) torch.Size([24, 100]) torch.Size([1, 13, 100])
hello ji  torch.Size([24, 100]) torch.Size([24, 100]) torch.Size([1, 13, 100])
hello ji  torch.Size([24, 100]) torch.Size([24, 100]) torch.Size([1, 13, 100])
input to network  torch.Size([1, 13, 100])
api is  torch.Size([1, 1, 100])
dims fadd ip torch.Size([1, 13, 100]) torch.Size([1, 1, 100]) torch.Size([1, 14, 100])
global readout shapes  torch.Size([1, 1, 14, 100]) torch.Size([1, 1, 14, 100]) torch.Size([1, 1, 100])
final shape  torch.Size([986])


tensor([ 1.4609e+00,  6.6074e-01,  4.6993e-01,  2.9189e-01,  8.5182e-01,
        -5.9710e-02, -2.6784e-02,  5.0952e-01,  5.3152e-01, -2.4074e-01,
        -9.9794e-01,  7.3011e-01,  8.8378e-02, -5.9137e-01,  4.2911e-01,
        -3.6567e-01, -8.8268e-01, -8.2736e-01,  5.0548e-02, -9.2682e-01,
         1.8905e-01, -3.1485e-01,  8.8396e-01,  5.2260e-01, -8.4950e-01,
        -7.3713e-01, -1.1603e+00,  1.5101e+00,  2.0267e-01, -1.5242e+00,
        -9.2259e-01,  6.3304e-01,  1.5189e+00, -1.4491e+00, -9.2293e-01,
        -1.4736e+00, -8.7386e-01, -2.2655e-01, -6.8803e-01, -1.0522e+00,
        -1.4521e+00,  3.6297e-01, -1.0983e+00, -8.6399e-01, -2.2681e-01,
        -4.6003e-01, -6.7433e-01, -2.4988e-01,  3.9334e-02,  1.5194e-01,
         1.3868e+00,  2.8630e-01,  4.2581e-01, -1.2784e+00,  8.9422e-02,
         7.8243e-01,  3.6766e-01,  3.2297e-01, -8.6034e-01,  6.3985e-01,
         8.5803e-01, -3.2008e-01,  6.2596e-01,  4.5191e-02,  2.1000e-01,
         1.6797e-02, -8.0690e-01, -1.4507e+00, -1.1

In [75]:
# from graphinvent.gnn.modules import GlobalReadout 
class MNN(SummationMPNN):
    def __init__(self,constants) -> None:
        super().__init__(constants)
        self.constants       = constants
        print(self.constants.message_size,self.constants.hidden_node_features,4)
        message_weights      = torch.Tensor(self.constants.message_size,
                                            self.constants.hidden_node_features,
                                            4)#edge features
        print(message_weights.shape)
        if False:#"cuda" == "cuda":
            message_weights = message_weights.to("cuda", non_blocking=True)
        
    
        self.message_weights = torch.nn.Parameter(message_weights)

        self.gru             = torch.nn.GRUCell(
            input_size=self.constants.message_size,
            hidden_size=self.constants.hidden_node_features,
            bias=True
        )
        
        self.APDReadout = GlobalReadout()

        self.reset_parameters()

    def reset_parameters(self) -> None:
        import math
        stdev = 1.0 / math.sqrt(self.message_weights.size(1))
        self.message_weights.data.uniform_(-stdev, stdev)

    def message_terms(self, nodes : torch.Tensor, node_neighbours : torch.Tensor,
                        edges : torch.Tensor) -> torch.Tensor:
        
        edges_view            = edges.view(-1, 1, 1, self.constants.n_edge_features)
        #print("edges ",edges_view.shape)
        weights_for_each_edge = (edges_view * self.message_weights.unsqueeze(0)).sum(3)
        return torch.matmul(weights_for_each_edge,
                            node_neighbours.unsqueeze(-1)).squeeze()
    #torch.broadcast_to(x, (3, 3))
    def update(self, nodes : torch.Tensor, messages : torch.Tensor) -> torch.Tensor:
        return self.gru(messages, nodes)

    def readout(self, hidden_nodes : torch.Tensor, input_nodes : torch.Tensor,
                node_mask : torch.Tensor) -> torch.Tensor:
        # graph_embeddings = torch.sum(hidden_nodes, dim=1)
        print("input to network ",hidden_nodes.shape)
        output           = self.APDReadout(hidden_nodes)
        return output



In [14]:
ash = molecule.get_decoding_APD()
ash.shape

NameError: name 'molecule' is not defined

In [72]:
# 

In [15]:
katai.shape

torch.Size([1, 23, 100])

In [89]:
edges.shape

torch.Size([1, 10, 10, 4])

In [74]:
class GlobalReadout(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.mlp1 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlp2 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp2_hidden_dim]*constants.mlp2_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        
        self.mlp3 = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlp4 = MLP(in_features=2*constants.message_size,
                  hidden_layer_sizes=[constants.mlp2_hidden_dim]*constants.mlp2_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.mlpt = MLP(in_features=constants.message_size,
                  hidden_layer_sizes=[constants.mlp1_hidden_dim]*constants.mlp1_depth,
                  out_features=constants.message_size,
                  dropout_p=0.0)
        self.final_l = MLP(in_features=2900,#23*100 #(14+14+1)100
                  hidden_layer_sizes=[500]*1,
                  out_features=989,
                  dropout_p=0.0)
          
    def forward(self,features):
        g= torch.sum(features,dim=1)
        g = g.view(1,1,100)
        #g = torch.broadcast_to(g, (1,10, 100))
        print("api is ",g.shape)
        fadd1 = self.mlp1(features)
        fconn1 = self.mlp2(features)  

        print("dims fadd ip",fconn1.shape,g.shape,torch.cat([fadd1,g],dim=1).shape)
        fadd = self.mlp3(torch.cat([fadd1,g],dim=1)).unsqueeze(dim=1)
        fconn = self.mlp3(torch.cat([fconn1,g],dim=1)).unsqueeze(dim=1)

        fterm = self.mlpt(g)
        print("global readout shapes ",fadd.shape,fconn.shape,fterm.shape)
        cat = torch.cat((fadd.squeeze(dim=1), fconn.squeeze(dim=1), fterm), dim=1)
        cat = torch.flatten(cat)
        cat = self.final_l(cat)
        print("final shape ",cat.shape)

        return cat
        #apd = self.Softmax()....from original code its removed

    
            

In [12]:

class MLP(torch.nn.Module):
    def __init__(self, in_features : int, hidden_layer_sizes : list, out_features : int,
                 dropout_p : float) -> None:
        super().__init__()

        activation_function = torch.nn.SELU
        # create list of all layer feature sizes
        fs = [in_features, *hidden_layer_sizes, out_features]
        # create list of linear_blocks
        layers = [self._linear_block(in_f, out_f,
                                     activation_function,
                                     dropout_p)
                  for in_f, out_f in zip(fs, fs[1:])]
        # concatenate modules in all sequentials in layers list
        layers = [module for sq in layers for module in sq.children()]

        # add modules to sequential container
        self.seq = torch.nn.Sequential(*layers)

    def _linear_block(self, in_f : int, out_f : int, activation : torch.nn.Module,
                      dropout_p : float) -> torch.nn.Sequential:
        
        # bias must be used in most MLPs in our models to learn from empty graphs
        linear = torch.nn.Linear(in_f, out_f, bias=True)
        torch.nn.init.xavier_uniform_(linear.weight)
        return torch.nn.Sequential(linear, activation(), torch.nn.AlphaDropout(dropout_p))

    def forward(self, layers_input : torch.nn.Sequential) -> torch.nn.Sequential:
        return self.seq(layers_input)


In [47]:
linear = torch.nn.Linear(100,1000, bias=True)


In [50]:
ip = torch.zeros((10,100))

In [51]:
node.shape
edge.shape

(10, 10, 4)

In [52]:
linear(ip).shape

torch.Size([10, 1000])